# Setup

In [1]:
import contracts
contracts.disable_all()

In [2]:
import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_html

INFO:dt-world:duckietown-world 5.0.10
INFO:zuper-typing:zuper-typing 5.0.12
INFO:zuper-commons:zuper-commons 5.0.7


In [3]:
dw.logger.setLevel(50)

Better visualization of output

In [4]:
%%html
<style>
pre {line-height: 90%}
</style>

# Road Network
Let's load a map and compute the road network.

In [5]:
m = dw.load_map('robotarium1')


Use the function `get_skeleton_graph`:

In [6]:
sk = dw.get_skeleton_graph(m)

The return type is `SkeletonGraphResult`. It contains in `sk.root2` a new map with the joined lane segments. 

In [7]:
ipython_draw_html(sk.root2);

While in the attribute `sk.G` we find a graph describing the topology.

This is a graph where each node is a meeting point between lanes, and each edge represents a lane.

In [8]:
# nodes
print(list(sk.G))

['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37', 'P38', 'P39', 'P40', 'P41', 'P42', 'P43', 'P44', 'P45', 'P46', 'P47', 'P48', 'P49', 'P50', 'P51', 'P52', 'P53', 'P54', 'P55', 'P56', 'P57', 'P58', 'P59', 'P60', 'P61', 'P62', 'P63', 'P64', 'P65', 'P66', 'P67', 'P68', 'P69', 'P70', 'P71', 'P72', 'P73', 'P74', 'P75', 'P76', 'P77', 'P78', 'P79', 'P80', 'P81', 'P82', 'P83']


In [9]:
# edges
for n1, n2 in sk.G.edges():
    data = sk.G.get_edge_data(n1, n2)
    one_lane = data[0]['lane']
    print('I can go from %s to %s using lane %s' % (n1, n2, one_lane))

I can go from P0 to P3 using lane L16
I can go from P1 to P4 using lane ls020
I can go from P1 to P5 using lane ls021
I can go from P2 to P1 using lane L17
I can go from P3 to P11 using lane ls043
I can go from P3 to P9 using lane ls042
I can go from P4 to P13 using lane ls045
I can go from P5 to P8 using lane L26
I can go from P6 to P0 using lane ls023
I can go from P6 to P4 using lane ls022
I can go from P7 to P0 using lane ls025
I can go from P7 to P5 using lane ls024
I can go from P8 to P11 using lane ls041
I can go from P8 to P2 using lane ls040
I can go from P9 to P6 using lane L27
I can go from P10 to P9 using lane ls039
I can go from P10 to P2 using lane ls038
I can go from P11 to P16 using lane L41
I can go from P12 to P7 using lane ls044
I can go from P13 to P14 using lane ls048
I can go from P13 to P15 using lane ls049
I can go from P14 to P19 using lane L43
I can go from P15 to P10 using lane L40
I can go from P16 to P12 using lane ls051
I can go from P16 to P14 using lane 

Let's bring in the `draw_graph` function from some time ago.

In [10]:
def draw_graph(G0, pos=None):
    import networkx as nx
    from matplotlib import pyplot as plt
    pos = pos or nx.spring_layout(G0)
    plt.figure(figsize=(12, 12))    
    nx.draw(G0,pos,labels={node:node for node in G0.nodes()})
    def edge_label(a, b):
        datas = G0.get_edge_data(a, b)
        s = '%d edge%s' % (len(datas), 's' if len(datas)>=2 else '')
        for k, v in datas.items():
            if v:
                if 'label' in v:
                    s += '\n %s' % v['label']
                else:
                    s += '\n %s' %v
        return s
    edge_labels = dict([ ((a,b), edge_label(a,b)) for a,b in G0.edges()])
    nx.draw_networkx_edge_labels(G0,pos,edge_labels=edge_labels,font_color='red')
    plt.axis('off')
    plt.show()

Set the position of each node in the graph based on the attribute 'point'.

In [11]:
import geometry as geo
pos = {}
for n in sk.G:
    q = sk.G.nodes[n]['point'].as_SE2()
    t, _ = geo.translation_angle_from_SE2(q)
    pos[n] = t
draw_graph(sk.G, pos=pos)

<Figure size 1200x1200 with 1 Axes>

## Planning example

Here is an example of how to do planning on the road network.

We select a start and end node:

In [12]:
start = 'P60'
end = 'P36'

We find the shortest path:

In [13]:
import networkx as nx
path = nx.shortest_path(sk.G, start, end)
print(path)

['P60', 'P57', 'P54', 'P46', 'P33', 'P31', 'P34', 'P36']


We retrieve the edge names:

In [14]:
def get_lanes(path):
    edges = zip(path[:-1], path[1:]) 
    lanes  = []
    for a, b in edges:
        lane = sk.G.get_edge_data(a, b)[0]['lane']
        lanes.append(lane)
    return lanes

In [15]:
lanes = get_lanes(path);
print(lanes)

['ls202', 'L111', 'ls193', 'L100', 'ls132', 'L86', 'ls140']


For visualization, we create a new map containing only the lanes selected: 

In [16]:
po = dw.PlacedObject()
for lane_name in lanes:
    lane = sk.root2.children[lane_name]
    po.set_object(lane_name, lane, ground_truth=dw.SE2Transform.identity())

In [17]:
ipython_draw_html(po);